# 2.1 Obtención de datos mediante APIs

In [1]:
import requests
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from unidecode import unidecode

In [58]:
# Set up the WebDriver
driver = webdriver.Chrome()

# Open the Python website
driver.get(
    "https://cuentame.inegi.org.mx/monografias/informacion/oax/territorio/div_municipal.aspx?tema=me&e=20"
)

# Wait to url is ready
time.sleep(5)

# Read and save information
oax_states = []
for i in range(1, 571):
    state = driver.find_element(
        By.XPATH, "//*[@id='keywords2']/tbody/tr[" + str(i) + "]/td[2]"
    )
    # Save and clean string
    oax_states.append(unidecode(state.text))

# Close the browser
driver.quit()

In [59]:
oax_states

['Abejones',
 'Acatlan de Perez Figueroa',
 'Asuncion Cacalotepec',
 'Asuncion Cuyotepeji',
 'Asuncion Ixtaltepec',
 'Asuncion Nochixtlan',
 'Asuncion Ocotlan',
 'Asuncion Tlacolulita',
 'Ayotzintepec',
 'El Barrio de la Soledad',
 'Calihuala',
 'Candelaria Loxicha',
 'Cienega de Zimatlan',
 'Ciudad Ixtepec',
 'Coatecas Altas',
 'Coicoyan de las Flores',
 'La Compania',
 'Concepcion Buenavista',
 'Concepcion Papalo',
 'Constancia del Rosario',
 'Cosolapa',
 'Cosoltepec',
 'Cuilapam de Guerrero',
 'Cuyamecalco Villa de Zaragoza',
 'Chahuites',
 'Chalcatongo de Hidalgo',
 'Chiquihuitlan de Benito Juarez',
 'Heroica Ciudad de Ejutla de Crespo',
 'Eloxochitlan de Flores Magon',
 'El Espinal',
 'Tamazulapam del Espiritu Santo',
 'Fresnillo de Trujano',
 'Guadalupe Etla',
 'Guadalupe de Ramirez',
 'Guelatao de Juarez',
 'Guevea de Humboldt',
 'Mesones Hidalgo',
 'Villa Hidalgo',
 'Heroica Ciudad de Huajuapan de Leon',
 'Huautepec',
 'Huautla de Jimenez',
 'Ixtlan de Juarez',
 'Juchitan de Za

Lista de estados de donde quiero obtener la información.

In [7]:
locations = [
    "Aguascalientes",
    "Campeche",
    "Chiapas",
    "Chihuahua",
    "Mexico",
    "Guerrero",
    "Jalisco",
    "Nuevo leon",
    "Oaxaca",
    "Puebla",
    "Queretaro",
    "Sinaloa",
    "Sonora",
    "Tamaulipas",
    "Veracruz",
    "Yucatan",
    "Zacatecas",
]

In [41]:
locations = [
    "Aguascalientes",
    "Baja California",
    "Baja California Sur",
    "Campeche",
    "Chiapas",
    "Chihuahua",
    "Ciudad de México",
    "Coahuila",
    "Colima",
    "Durango",
    "Estado de México",
    "Guanajuato",
    "Guerrero",
    "Hidalgo",
    "Jalisco",
    "Michoacán",
    "Morelos",
    "Nayarit",
    "Nuevo León",
    "Oaxaca",
    "Puebla",
    "Querétaro",
    "Quintana Roo",
    "San Luis Potosí",
    "Sinaloa",
    "Sonora",
    "Tabasco",
    "Tamaulipas",
    "Tlaxcala",
    "Veracruz",
    "Yucatán",
    "Zacatecas",
]

In [ ]:
len(locations)

17

## Envío de solicitudes a la API

In [9]:
def getApiData(endpoint, payload, locations, getFirst):
    data_collected = []
    for location in locations:
        payload["q"] = location
        try:
            time.sleep(5)
            r = requests.get(url=endpoint, params=payload)
            if r.status_code == 200:
                json = r.json()
                if getFirst:
                    data_collected.append(json[0])
                else:
                    data_collected.append(json)
        except:
            continue
    return data_collected

In [10]:
endpoint = "http://api.weatherapi.com/v1/"
payload = {"key": "cf111e82db2c47119ea162327242212"}

In [ ]:
r = requests.get(url=endpoint, params=payload)

In [11]:
search = "/search.json"

In [42]:
data_search = getApiData(
    endpoint=endpoint + search, payload=payload, locations=locations, getFirst=True
)

In [13]:
current = "/current.json"

In [14]:
data_current = getApiData(
    endpoint=endpoint + current, payload=payload, locations=locations, getFirst=False
)

In [15]:
astronomy = "/astronomy.json"

In [16]:
data_astronomy = getApiData(
    endpoint=endpoint + astronomy, payload=payload, locations=locations, getFirst=False
)

In [43]:
normalized_data_search = pd.json_normalize(data_search)
df_search_temp = pd.DataFrame.from_dict(normalized_data_search).add_prefix("location.")
df_search = df_search_temp.drop(["location.lat", "location.lon"], axis=1)
df_search

,location.id,location.name,location.region,location.country,location.url
0,3296893,Aguascalientes,Aguascalientes,Mexico,aguascalientes-aguascalientes-mexico
1,3242121,Baja California,Chiapas,Mexico,baja-california-chiapas-mexico
2,3274846,Agrodelicias De La Baja Sur,Baja California Sur,Mexico,agrodelicias-de-la-baja-sur-baja-california-su...
3,3220121,Campeche,Campeche,Mexico,campeche-campeche-mexico
4,3241759,Chiapas Nuevo,Chiapas,Mexico,chiapas-nuevo-chiapas-mexico
5,3295284,Chihuahua,Chihuahua,Mexico,chihuahua-chihuahua-mexico
6,3198637,Ciudad Jardin,México,Mexico,ciudad-jardin-mxico-mexico
7,3182624,Coahuila,San Luis PotosÃ­,Mexico,coahuila-san-luis-potos-mexico
8,3325202,Colima,Colima,Mexico,colima-colima-mexico
9,707103,Durango,Pais Vasco,Spain,durango-pais-vasco-spain


In [18]:
normalized_data_current = pd.json_normalize(data_current)
df_current = pd.DataFrame.from_dict(normalized_data_current)
df_current.head()

,location.name,location.region,location.country,location.lat,location.lon,location.tz_id,location.localtime_epoch,location.localtime,current.last_updated_epoch,current.last_updated,...,current.windchill_f,current.heatindex_c,current.heatindex_f,current.dewpoint_c,current.dewpoint_f,current.vis_km,current.vis_miles,current.uv,current.gust_mph,current.gust_kph
0,Aguascalientes,Aguascalientes,Mexico,21.8823,-102.2826,America/Mexico_City,1735524907,2024-12-29 20:15,1735524900,2024-12-29 20:15,...,66.5,18.9,65.9,-7.4,18.6,16.0,9.0,0.0,10.3,16.6
1,Campeche,Campeche,Mexico,19.8380,-90.5277,America/Merida,1735527039,2024-12-29 20:50,1735526700,2024-12-29 20:45,...,73.1,25.3,77.6,22.0,71.6,10.0,6.0,0.0,13.8,22.2
2,Chiapas Nuevo,Chiapas,Mexico,16.4553,-93.8272,America/Mexico_City,1735526990,2024-12-29 20:49,1735526700,2024-12-29 20:45,...,68.8,20.9,69.7,18.2,64.7,10.0,6.0,0.0,9.2,14.8
3,Chihuahua,Chihuahua,Mexico,28.6353,-106.0889,America/Chihuahua,1735527016,2024-12-29 20:50,1735526700,2024-12-29 20:45,...,61.5,16.4,61.6,-10.4,13.4,16.0,9.0,0.0,7.5,12.1
4,Mexico City,The Federal District,Mexico,19.4290,-99.1280,America/Mexico_City,1735526993,2024-12-29 20:49,1735526700,2024-12-29 20:45,...,64.8,18.3,64.9,-1.6,29.0,14.0,8.0,0.0,5.1,8.3


In [19]:
normalized_data_astronomy = pd.json_normalize(data_astronomy)
df_astronomy = pd.DataFrame.from_dict(normalized_data_astronomy)
df_astronomy.head()

,location.name,location.region,location.country,location.lat,location.lon,location.tz_id,location.localtime_epoch,location.localtime,astronomy.astro.sunrise,astronomy.astro.sunset,astronomy.astro.moonrise,astronomy.astro.moonset,astronomy.astro.moon_phase,astronomy.astro.moon_illumination,astronomy.astro.is_moon_up,astronomy.astro.is_sun_up
0,Aguascalientes,Aguascalientes,Mexico,21.8823,-102.2826,America/Mexico_City,1735527064,2024-12-29 20:51,07:28 AM,06:16 PM,07:24 AM,06:08 PM,New Moon,1,0,0
1,Campeche,Campeche,Mexico,19.8380,-90.5277,America/Merida,1735527078,2024-12-29 20:51,06:36 AM,05:34 PM,06:29 AM,05:24 PM,New Moon,1,0,0
2,Chiapas Nuevo,Chiapas,Mexico,16.4553,-93.8272,America/Mexico_City,1735526364,2024-12-29 20:39,06:43 AM,05:53 PM,06:34 AM,05:46 PM,New Moon,1,0,0
3,Chihuahua,Chihuahua,Mexico,28.6353,-106.0889,America/Chihuahua,1735527137,2024-12-29 20:52,07:57 AM,06:18 PM,07:59 AM,06:05 PM,New Moon,1,0,0
4,Mexico City,The Federal District,Mexico,19.4290,-99.1280,America/Mexico_City,1735527094,2024-12-29 20:51,07:10 AM,06:09 PM,07:04 AM,06:01 PM,New Moon,1,0,0


In [37]:
df_astronomy_current = pd.merge(
    df_astronomy,
    df_current,
    on=[
        "location.name",
        "location.region",
        "location.country",
        "location.lat",
        "location.lon",
        "location.tz_id",
    ],
)
df_astronomy_current

,location.name,location.region,location.country,location.lat,location.lon,location.tz_id,location.localtime_epoch_x,location.localtime_x,astronomy.astro.sunrise,astronomy.astro.sunset,...,current.windchill_f,current.heatindex_c,current.heatindex_f,current.dewpoint_c,current.dewpoint_f,current.vis_km,current.vis_miles,current.uv,current.gust_mph,current.gust_kph
0,Aguascalientes,Aguascalientes,Mexico,21.8823,-102.2826,America/Mexico_City,1735527064,2024-12-29 20:51,07:28 AM,06:16 PM,...,66.5,18.9,65.9,-7.4,18.6,16.0,9.0,0.0,10.3,16.6
1,Campeche,Campeche,Mexico,19.8380,-90.5277,America/Merida,1735527078,2024-12-29 20:51,06:36 AM,05:34 PM,...,73.1,25.3,77.6,22.0,71.6,10.0,6.0,0.0,13.8,22.2
2,Chiapas Nuevo,Chiapas,Mexico,16.4553,-93.8272,America/Mexico_City,1735526364,2024-12-29 20:39,06:43 AM,05:53 PM,...,68.8,20.9,69.7,18.2,64.7,10.0,6.0,0.0,9.2,14.8
3,Chihuahua,Chihuahua,Mexico,28.6353,-106.0889,America/Chihuahua,1735527137,2024-12-29 20:52,07:57 AM,06:18 PM,...,61.5,16.4,61.6,-10.4,13.4,16.0,9.0,0.0,7.5,12.1
4,Mexico City,The Federal District,Mexico,19.4290,-99.1280,America/Mexico_City,1735527094,2024-12-29 20:51,07:10 AM,06:09 PM,...,64.8,18.3,64.9,-1.6,29.0,14.0,8.0,0.0,5.1,8.3
5,Guerrero,Hidalgo,Mexico,20.3669,-99.0075,America/Mexico_City,1735527148,2024-12-29 20:52,07:12 AM,06:06 PM,...,55.6,13.6,56.5,8.3,46.9,10.0,6.0,0.0,14.1,22.7
6,Jalisco Del Refugio (Jalisco),Jalisco,Mexico,21.9911,-103.2340,America/Mexico_City,1735525033,2024-12-29 20:17,07:32 AM,06:20 PM,...,60.4,15.8,60.5,-10.5,13.1,10.0,6.0,0.0,4.7,7.6
7,Nuevo Leon,Chiapas,Mexico,16.9403,-91.7127,America/Mexico_City,1735527102,2024-12-29 20:51,06:35 AM,05:44 PM,...,63.9,18.0,64.3,16.8,62.2,0.0,0.0,0.0,3.3,5.3
8,Oaxaca,Oaxaca,Mexico,17.0711,-96.7191,America/Mexico_City,1735527108,2024-12-29 20:51,06:56 AM,06:04 PM,...,56.4,14.0,57.3,5.4,41.6,10.0,6.0,0.0,9.4,15.1
9,Puebla,Puebla,Mexico,19.0433,-98.2019,America/Mexico_City,1735525049,2024-12-29 20:17,07:06 AM,06:06 PM,...,64.4,18.2,64.7,1.0,33.9,16.0,9.0,0.0,5.6,9.1


In [39]:
df_full = pd.merge(
    df_search,
    df_astronomy_current,
    on=[
        "location.name",
        "location.region",
        "location.country",
    ],
)

In [40]:
df_full.to_csv("dataset/dataset_raw/weather.csv")